In [1]:
import numpy as np
import pandas as pd
import pprint 
import logging

from haystack import Finder
from haystack.utils import print_answers
from haystack.reader.farm import FARMReader
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever

11/12/2020 12:28:49 - INFO - faiss -   Loading faiss.


In [21]:
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True,
})

In [5]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

11/12/2020 12:37:16 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.010s]
11/12/2020 12:37:16 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.005s]
11/12/2020 12:37:16 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.030s]
11/12/2020 12:37:16 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.005s]


In [3]:
document_store.get_document_count()

11/11/2020 08:14:37 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.103s]


15891

In [7]:
retriever = ElasticsearchRetriever(document_store=document_store)

In [5]:
reader = FARMReader("ktrapeznikov/biobert_v1.1_pubmed_squad_v2", use_gpu=False)

11/11/2020 08:15:10 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/11/2020 08:15:10 - INFO - farm.infer -   Could not find `ktrapeznikov/biobert_v1.1_pubmed_squad_v2` locally. Try to download from model hub ...
11/11/2020 08:15:14 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/11/2020 08:15:20 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/11/2020 08:15:20 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
11/11/2020 08:15:20 - INFO - farm.infer -    0    0    0    0    0    0    0 
11/11/2020 08:15:20 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\
11/11/2020 08:15:20 - INFO - farm.infer -   /'\  / \  /'\  /'\  / \  / \  /'\
11/11

In [8]:
finder = Finder(reader, retriever)

In [9]:
nf_faq = pd.read_csv("../data/NF FAQ.csv")

In [10]:
nf_faq.head()

,Question,Answer
0,What is Neurofibromatosis?,NaN
1,How many people have Neurofibromatosis?,NaN
2,Is neurofibromatosis contagious?,NaN
3,Why do I have cafe au lait spots on my body?,NaN
4,Why do I have bumps under my skin?,NaN


In [13]:
nf_faq.shape

(54, 2)

In [18]:
def get_answers(preds):
    all_answers = []
    for ap in preds['answers']:
        tmp = {}
        tmp['answer'] = ap.get('answer', '')
        tmp['context'] = ap.get('context', '')
        tmp['document_id'] = ap.get('document_id', '')
        meta_dict = ap.get('meta', None)
        if meta_dict:
            tmp['title'] = meta_dict.get('title', '')
            tmp['keywords'] = meta_dict.get('keywords', '')
        all_answers.append(tmp)
    pprint.pprint(all_answers)

In [19]:
for i, row in nf_faq.iterrows():
    question_txt = row['Question']
    print(f"Question: {question_txt}")
    prediction = finder.get_answers(question=question_txt, top_k_retriever=10, top_k_reader=5)
    get_answers(prediction)
    print("\n")

11/11/2020 08:51:08 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.016s]
11/11/2020 08:51:08 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 08:51:08 - INFO - haystack.finder -   Reader is looking for detailed answer in 11639 chars ...
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Question: What is Neurofibromatosis?


Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.93s/ Batches]
11/11/2020 08:51:22 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.031s]
11/11/2020 08:51:22 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 08:51:22 - INFO - haystack.finder -   Reader is looking for detailed answer in 15403 chars ...
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

[{'answer': 'a common heritable neurocutaneous disorder',
  'context': 'Introduction: Neurofibromatosis type 1 is a common heritable '
             'neurocutaneous disorder. Neurofibromatosis type 1 may be '
             'associated with tumors of the ce',
  'document_id': '077b7779-0ccd-4900-b6bb-92295104da20',
  'keywords': '',
  'title': 'Neurofibromatosis type 1 associated with papillary thyroid '
           'carcinoma incidentally detected by thyroid ultrasonography: A case '
           'report'},
 {'answer': 'overactivation of the RAS-MAPK pathway',
  'context': 'nown and objective Neurofibromatosis is associated with '
             'overactivation of the RAS-MAPK pathway. MEK inhibitors have been '
             'shown to be an effective trea',
  'document_id': '1326a9de-cb78-443a-b851-853ad414cf9b',
  'keywords': 'medication pharmacogenetics',
  'title': 'Prolonged disease control with MEK inhibitor in neurofibromatosis '
           'type I-associated glioblastoma'},
 {'answer':

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.64 Batches/s]
11/11/2020 08:51:41 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.018s]
11/11/2020 08:51:41 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 08:51:41 - INFO - haystack.finder -   Reader is looking for detailed answer in 9757 chars ...
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

...

[{'answer': 'high prevalence of ASD symptoms',
  'context': 've population-based sample of children with NF1 indicates a high '
             'prevalence of ASD symptoms associated with NF1 as well as '
             'substantial co-occurrence w',
  'document_id': '53ca25c0-40c0-40b0-9d57-aa106b118e47',
  'keywords': '',
  'title': 'Autism and other psychiatric comorbidity in neurofibromatosis type '
           '1: Evidence from a population-based study'},
 {'answer': '80% of affected children exhibit cognitive and behavioral issues',
  'context': 'used by mutation of the NF1 gene, in which 80% of affected '
             'children exhibit cognitive and behavioral issues. Based on '
             'emerging evidence that NF1 may b',
  'document_id': 'e078992d-d253-4847-b37a-4a6320735132',
  'keywords': 'autism communication mouse neurofibromatosis type 1 ultrasonic '
              'vocalization',
  'title': 'Characterization of early communicative behavior in mouse models '
           'of n

Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.24s/ Batches]
11/11/2020 09:04:02 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
11/11/2020 09:04:02 - INFO - haystack.finder -   Got 10 candidates from retriever
11/11/2020 09:04:02 - INFO - haystack.finder -   Reader is looking for detailed answer in 12058 chars ...
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

[{'answer': 'will never be able to walk physiologically',
  'context': 'propriate treatment, a child with congenital clubfoot will never '
             'be able to walk physiologically with a dramatic impact on the '
             'quality of life. In the',
  'document_id': '726650e8-756e-4959-9bff-cbd2a68f9a71',
  'keywords': 'Clubfoot, Talipes, Equinovarus, Management, Pediatric',
  'title': 'What a paediatrician should know about congenital clubfoot'},
 {'answer': 'The karyotypes of the child and his parents were all normal',
  'context': 'ide, and irregular solid nodules on the back. The karyotypes of '
             'the child and his parents were all normal, while aCGH has '
             'identified a de novo constit',
  'document_id': '2f50c562-425b-4533-be25-a150330cdf20',
  'keywords': '17q11. 2 microdeletion Array comparative genomic hybridization '
              'Neurofibromatosis type 1',
  'title': 'Phenotypic and genetic analysis of a child carrying a 17q11. 2 '
           

Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.18s/ Batches]

[{'answer': 'No significant differences were observed on any of the measures '
            'of speech and language for children with non-familial versus '
            'familial NF1',
  'context': 'oblem. No significant differences were observed on any of the '
             'measures of speech and language for children with non-familial '
             'versus familial NF1. Resu',
  'document_id': 'be6dd3d9-b488-4fcd-8c69-c9fcb43867fe',
  'keywords': 'Articulation Children Language NF1 Neurofibromatosis type 1 '
              'Preschool Resonance Speech Voice',
  'title': 'Speech-language characteristics of children with neurofibromatosis '
           'type 1'},
 {'answer': 'deviations',
  'context': 'vious studies have shown that NF1 may be associated with a '
             "variety of deviations in the patients' speech, they have not "
             'investigated the characteristi',
  'document_id': '80da104f-946d-484a-9347-24aab63bcb35',
  'keywords': 'Articulation Fluency Neurofibromat

In [23]:
prediction = finder.get_answers(question='How often gliomas become malignant in NF1', top_k_retriever=6, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.32 Batches/s]


In [24]:
prediction

{'question': 'How often gliomas become malignant in NF1',
 'no_ans_gap': 3.3972415924072266,
 'answers': [{'answer': '1.4%',
   'score': 6.517305374145508,
   'probability': 0.6931023067482888,
   'context': 'reading to hypothalamus. Spontaneous regression was documented in 4/285 (1.4%) gliomas and the same number of NF1 children died due to gliomas. Obstru',
   'offset_start': 73,
   'offset_end': 77,
   'offset_start_in_doc': 893,
   'offset_end_in_doc': 897,
   'document_id': '7aae914f-58cb-45bf-89d2-267b8c1de4f5',
   'meta': {'title': 'Brain gliomas, hydrocephalus and idiopathic aqueduct stenosis in children with neurofibromatosis type 1',
    'doi': '10.1016/j.braindev.2019.04.003',
    'keywords': 'Brain glioma Hydrocephalus Idiopathic aqueduct stenosis Neurofibromatosis type 1 Optic pathway glioma'}},
  {'answer': 'the majority of gliomas arising in adults with NF1 are malignant cancers, typically glioblastoma, involving the cerebral hemispheres',
   'score': 5.096111297607422,


In [31]:
paragraph = prediction['answers'][2]['context']
paragraph[prediction['answers'][2]['offset_start']: prediction['answers'][2]['offset_end']]

'About 15%'

In [34]:
from annotated_text import annotated_text

## Test API

In [37]:
import json
import requests 

In [38]:
url = "http://127.0.0.1:8000/models/1/doc-qa"
payload = {
  "questions": [
    "How often gliomas become malignant in NF1"
  ],
  "top_k_reader": 3,
  "top_k_retriever": 6
}

In [39]:
result = requests.post(url, data=json.dumps(payload))

In [40]:
result

<Response [200]>

In [46]:
result.status_code

200

In [56]:
json.loads(result.text)['results'][0]

{'question': 'How often gliomas become malignant in NF1',
 'answers': [{'answer': 'the majority of gliomas arising in adults with NF1 are malignant cancers, typically glioblastoma, involving the cerebral hemispheres',
   'score': 5.096111297607422,
   'probability': 0.6540781398101104,
   'context': 'ldhood, gliomas affecting the optic pathway are most frequently encountered, whereas gliomas of the brainstem and other locations are observed in slightly older children. In contrast, the majority of gliomas arising in adults with NF1 are malignant cancers, typically glioblastoma, involving the cerebral hemispheres. Our understanding of the pathogenesis of NF1-associated gliomas has been significantly advanced through the use of genetically engineered mice, yielding new targets for therapeutic d',
   'offset_start': 184,
   'offset_end': 316,
   'offset_start_in_doc': 503,
   'offset_end_in_doc': 635,
   'document_id': '20f9fb49-9324-4a61-8539-49b735b1fa5f',
   'meta': {'keywords': 'brain 

In [55]:
def get_answers(preds):
    all_answers = []
    for ap in preds['answers']:
        tmp = {}
        tmp['answer'] = ap.get('answer', '')
        tmp['context'] = ap.get('context', '')
        tmp['offset_start'] = ap.get('offset_start', 0)
        tmp['offset_end'] = ap.get('offset_end', 0)
        tmp['document_id'] = ap.get('document_id', '')
        meta_dict = ap.get('meta', None)
        if meta_dict:
            tmp['title'] = meta_dict.get('title', '')
            tmp['keywords'] = meta_dict.get('keywords', '')
        all_answers.append(tmp)
    #pprint.pprint(all_answers)
    return all_answers

In [59]:
answers = get_answers(json.loads(result.text)['results'][0])

In [58]:
def show_annotation_text(answers):
    title = answers.get('title',"")
    keywords = answers.get('keywords',"")
    show_keywords_str = f"**Keywords**: {keywords} \n" if keywords!='' else '' 
    paragraph = answers['context']
    start_idx = answers['offset_start']
    end_idx = answers['offset_end']
    return annotated_text(
        '**Title**: ' + title + '\n',
        show_keywords_str,
        paragraph[:start_idx],
        (paragraph[start_idx:end_idx], 'answer' ,"#faa"),
        paragraph[end_idx:]
    )

In [64]:
show_annotation_text(answers[0])

In [68]:
answers[0]

{'answer': 'the majority of gliomas arising in adults with NF1 are malignant cancers, typically glioblastoma, involving the cerebral hemispheres',
 'context': 'ldhood, gliomas affecting the optic pathway are most frequently encountered, whereas gliomas of the brainstem and other locations are observed in slightly older children. In contrast, the majority of gliomas arising in adults with NF1 are malignant cancers, typically glioblastoma, involving the cerebral hemispheres. Our understanding of the pathogenesis of NF1-associated gliomas has been significantly advanced through the use of genetically engineered mice, yielding new targets for therapeutic d',
 'offset_start': 184,
 'offset_end': 316,
 'document_id': '20f9fb49-9324-4a61-8539-49b735b1fa5f',
 'title': 'Brain tumors in neurofibromatosis type 1',
 'keywords': 'brain tumor, brainstem, glioma, NF1, optic pathway, RAS'}